# Capstone Add-On: Impact Validation & Production Monitoring

**Prerequisite:** Complete the main capstone project first.

**Scenario:** You've built a churn prediction model with 3x+ lift. Leadership likes the results but asks:

> "How do we PROVE this actually reduces churn? And how do we keep it healthy after launch?"

This add-on covers:
1. Designing an A/B test or holdout experiment
2. Understanding why risk models can waste money (uplift vs. risk)
3. Building a production monitoring plan

**Runtime:** ~30-45 minutes

---

In [ ]:
# Setup (same as main capstone)
import pandas as pd
import numpy as np

# Load your capstone results (or re-run if needed)
# These are the key numbers from your main capstone:
BASE_CHURN_RATE = 0.12  # Update with your actual value
MODEL_PRECISION_AT_500 = 0.40  # Update with your actual value
LIFT = 3.3  # Update with your actual value

# Business parameters
CAPACITY = 500  # Calls per week
SAVE_RATE = 0.30  # 30% of churners can be saved by a call
LTV_PER_SAVE = 200  # $200 value per saved customer
COST_PER_CALL = 15  # $15 per call

print("Setup complete!")
print(f"Your model: {MODEL_PRECISION_AT_500:.0%} precision @ 500 ({LIFT:.1f}x lift)")

---

# Part 1: Experiment Design (Proving Business Impact)

Your offline metrics (AUC, Precision@500) prove the model **ranks well**. But they don't prove:
- That calling these customers actually saves them
- That your targeting is better than other strategies
- That the model will keep working over time

To prove business impact, you need an **experiment**.

## 1.1 Choosing an Experiment Design

Three common approaches:

| Design | How it works | Pros | Cons |
|--------|--------------|------|------|
| **A/B test** | Randomly assign customers to Model vs. Random targeting | Gold standard | Wastes calls on random group |
| **Holdout** | Give most customers to model, hold back small control | More efficient | Less statistical power |
| **Champion/Challenger** | Model vs. current production rule | Tests incremental improvement | Confounded if rule is correlated with model |

For StreamCart, we'll design a **holdout experiment** since the retention team doesn't want to waste too many calls.

In [ ]:
# TODO: Complete the experiment plan

experiment_plan = """
=== EXPERIMENT PLAN: CHURN MODEL IMPACT TEST ===

1. HYPOTHESIS
   If we use the churn model to target retention calls (instead of random),
   we will reduce churn by [X]% among the called population.
   
   YOUR ANSWER: _____________

2. PRIMARY METRIC
   What business outcome will you measure?
   (Hint: churn rate, saves, revenue... be specific about time window)
   
   YOUR ANSWER: _____________

3. POPULATION
   Who is eligible for this experiment?
   (Hint: tenure requirements? active subscribers only?)
   
   YOUR ANSWER: _____________

4. RANDOMIZATION UNIT
   What gets randomly assigned: customer, household, or something else?
   
   YOUR ANSWER: _____________

5. TREATMENT VS CONTROL
   - Treatment (____%): Model selects top [K] customers for calls
   - Control (____%): Random selection OR no calls
   
   YOUR ANSWER: Treatment ___%, Control ___%
   
   Why did you choose this split?
   YOUR ANSWER: _____________

6. DURATION
   How long will you run the experiment?
   (Consider: how long until churn manifests? sample size needed?)
   
   YOUR ANSWER: _____________

7. GUARDRAILS
   What could go wrong? How will you check for contamination?
   
   YOUR ANSWER: _____________

"""
print(experiment_plan)

## 1.2 Sample Size Considerations

Quick rule of thumb for a two-group comparison:

```
n ≈ 16 × σ² / δ²

where:
- σ = standard deviation of the outcome
- δ = minimum detectable effect (the difference you want to find)
```

For binary outcomes (churn/no churn), σ² ≈ p(1-p) where p is the base rate.

In [ ]:
# Sample size estimation

# Baseline churn rate (your estimate)
p_baseline = BASE_CHURN_RATE

# Minimum detectable effect (MDE)
# If model saves 30% of churners and reaches 3x more churners,
# expected churn reduction among called customers is roughly:
# (more churners reached) × (save rate) = potential improvement

# TODO: Calculate your expected churn reduction
# Hint: If random reaches 12% churners and model reaches 40%,
#       and 30% of reached churners are saved...

random_saves_per_100_calls = 100 * BASE_CHURN_RATE * SAVE_RATE
model_saves_per_100_calls = 100 * MODEL_PRECISION_AT_500 * SAVE_RATE

print(f"Random: {random_saves_per_100_calls:.1f} saves per 100 calls")
print(f"Model: {model_saves_per_100_calls:.1f} saves per 100 calls")
print(f"Expected improvement: {model_saves_per_100_calls - random_saves_per_100_calls:.1f} extra saves per 100 calls")

In [ ]:
# Rough sample size calculation
import math

# Variance for binary outcome
variance = p_baseline * (1 - p_baseline)

# Effect size we want to detect (e.g., 3 percentage points)
delta = 0.03  # TODO: Adjust based on your expected effect

# Sample size per group (rough rule of thumb)
n_per_group = 16 * variance / (delta ** 2)

print(f"To detect a {delta:.0%} difference with ~80% power:")
print(f"Need approximately {n_per_group:,.0f} customers per group")
print(f"\nWith 500 calls/week:")
print(f"  - 80/20 split: Treatment gets ~400/week, Control gets ~100/week")
print(f"  - Weeks needed: ~{n_per_group / 100:.0f} weeks for Control to reach sample size")

## 1.3 Self-Check: Experiment Design

In [ ]:
# Answer these questions to validate your experiment design

design_check = """
=== EXPERIMENT DESIGN CHECKLIST ===

□ I have a clear, measurable hypothesis
□ My primary metric is a business outcome (not a model metric)
□ I've defined who is eligible and who is excluded
□ I've specified how randomization will work
□ My Treatment/Control split makes sense given constraints
□ I've estimated how long the experiment needs to run
□ I've identified potential threats (contamination, spillover, etc.)

Common mistakes to avoid:
- Using AUC or Precision as the experiment outcome (these are model metrics, not business metrics)
- Running too short and getting noisy results
- Peeking at results early and stopping when it "looks good"
- Forgetting to check that Treatment and Control are balanced
"""
print(design_check)

---

# Part 2: Risk vs. Uplift (When Targeting Can Backfire)

Your churn model predicts **who is likely to churn**. But the business cares about **who we can save**.

These are not the same thing.

## 2.1 The Four Quadrants

```
                    Would Churn Without Call?
                         YES          NO
                    ┌───────────┬───────────┐
    Will Call       │ PERSUADE  │  WASTED   │
    Save Them?      │  (value)  │  EFFORT   │
         YES        │           │           │
                    ├───────────┼───────────┤
         NO         │   LOST    │ SLEEPING  │
                    │  CAUSE    │   DOGS    │
                    └───────────┴───────────┘
```

**Risk models find the left column** (likely to churn). But only the **top-left** creates value.

In [ ]:
# TODO: Think through this scenario

scenario = """
=== SCENARIO: TWO HIGH-RISK CUSTOMERS ===

Both customers have 70% predicted churn probability. Your model ranks them equally.

Customer A:
- Frustrated about a billing issue
- Recently contacted support twice
- Has been a customer for 2 years

Customer B:
- Already signed up with a competitor
- Hasn't logged in for 60 days
- Posted on social media about leaving

QUESTIONS:

1. If you can only call ONE customer, which would you choose?
   YOUR ANSWER: _____________

2. Why might calling Customer B be a waste of money?
   YOUR ANSWER: _____________

3. What additional signals could help distinguish Persuadables from Lost Causes?
   YOUR ANSWER: _____________

"""
print(scenario)

## 2.2 Break-Even Analysis

Even without uplift modeling, you can calculate the **break-even probability** threshold.

In [ ]:
# TODO: Calculate break-even threshold

# Expected value of calling a customer with churn probability P:
# EV = P × SaveRate × LTV - CostPerCall
#
# Break-even when EV = 0:
# P × SaveRate × LTV = CostPerCall
# P = CostPerCall / (SaveRate × LTV)

break_even_p = COST_PER_CALL / (SAVE_RATE * LTV_PER_SAVE)

print(f"=== BREAK-EVEN ANALYSIS ===")
print(f"Cost per call: ${COST_PER_CALL}")
print(f"Save rate: {SAVE_RATE:.0%}")
print(f"Value per save: ${LTV_PER_SAVE}")
print(f"")
print(f"Break-even churn probability: {break_even_p:.1%}")
print(f"")
print(f"Interpretation: Only call customers with P(churn) > {break_even_p:.1%}")

In [ ]:
# TODO: Answer this question

critical_thinking = """
=== CRITICAL THINKING ===

The break-even calculation assumes that the SAVE RATE is constant across all customers.

But is this realistic?

Consider:
- Does a "Lost Cause" customer have the same 30% save rate as a "Persuadable"?
- If not, how does this change your targeting strategy?

YOUR ANSWER:
_____________________________________________________________
_____________________________________________________________
_____________________________________________________________

"""
print(critical_thinking)

## 2.3 Practical Fallback: Hybrid Scoring

If you don't have data for true uplift modeling, layer **engagement signals** on top of risk.

In [ ]:
# Example: Hybrid scoring approach
# (This is a heuristic, not true uplift modeling)

# Simulated data for illustration
np.random.seed(42)
n_customers = 1000

df_example = pd.DataFrame({
    'customer_id': range(n_customers),
    'churn_prob': np.random.beta(2, 10, n_customers),  # Model output
    'logins_last_30d': np.random.poisson(5, n_customers),  # Engagement proxy
})

# Pure risk ranking
df_example['risk_rank'] = df_example['churn_prob'].rank(ascending=False)

# Hybrid: risk × engagement (normalized)
# Intuition: target customers who are at-risk AND still engaged (persuadable)
df_example['engagement_score'] = np.clip(df_example['logins_last_30d'] / 10, 0.1, 1.0)
df_example['hybrid_score'] = df_example['churn_prob'] * df_example['engagement_score']
df_example['hybrid_rank'] = df_example['hybrid_score'].rank(ascending=False)

# Compare top 50 by each method
print("=== TOP 50 COMPARISON ===")
print("\nPure Risk (top 5):")
top_risk = df_example.nsmallest(5, 'risk_rank')[['customer_id', 'churn_prob', 'logins_last_30d']]
print(top_risk.to_string(index=False))

print("\nHybrid Score (top 5):")
top_hybrid = df_example.nsmallest(5, 'hybrid_rank')[['customer_id', 'churn_prob', 'logins_last_30d']]
print(top_hybrid.to_string(index=False))

In [ ]:
# TODO: Explain the difference

hybrid_explanation = """
=== YOUR ANALYSIS ===

Look at the two rankings above.

1. What's different about the customers selected by Hybrid vs Pure Risk?
   YOUR ANSWER: _____________

2. Why might the Hybrid approach be better for intervention targeting?
   YOUR ANSWER: _____________

3. What's the main limitation of this heuristic?
   (Hint: we're not measuring actual treatment effect)
   YOUR ANSWER: _____________

"""
print(hybrid_explanation)

---

# Part 3: Production Monitoring Plan

Deploying the model is not the end-it's the beginning. Models decay, data drifts, and business changes.

## 3.1 What Can Go Wrong

| Problem | Symptom | Example |
|---------|---------|----------|
| **Data drift** | Feature distributions shift | Login behavior changes after app redesign |
| **Concept drift** | Same features, different outcomes | Competitor launches aggressive win-back campaign |
| **Silent failure** | Model runs but outputs garbage | Upstream data pipeline broke |
| **Feedback loop** | Model performance degrades | Saved customers change future training data |

In [ ]:
# TODO: Complete your monitoring plan

monitoring_plan = """
=== PRODUCTION MONITORING PLAN: CHURN MODEL ===

Model: StreamCart Churn Prediction
Owner: [Your Name]
Last Trained: [Date]

---

DAILY CHECKS (automated alerts)

□ Data freshness
  - Check: Features updated within last 24 hours
  - Alert if: Timestamp > 24h old
  - YOUR ACTION IF ALERT: _____________

□ Missing data
  - Check: Null rates for key features
  - Alert if: Null rate > [X]% (set your threshold)
  - YOUR THRESHOLD: _____________

□ Score distribution
  - Check: Are predictions still spread out?
  - Alert if: >80% of scores in single decile
  - WHY THIS MATTERS: _____________

---

WEEKLY CHECKS (manual review)

□ Feature distributions
  - Compare this week's distributions to historical
  - Flag if: Mean shifted >2 standard deviations
  - TOP 3 FEATURES TO WATCH:
    1. _____________
    2. _____________
    3. _____________

□ Precision@500
  - Calculate: Of top 500 scored customers, how many actually churned?
  - Baseline: [Your model's test set precision]
  - Alert if: Drops >15% from baseline
  - YOUR BASELINE VALUE: _____________

□ Lift vs random
  - Compare: Model precision vs base churn rate
  - Minimum acceptable lift: [Your threshold]
  - YOUR MINIMUM LIFT: _____________

---

MONTHLY CHECKS

□ Calibration
  - Check: Do predicted probabilities match actual rates?
  - Method: Calibration plot (predicted vs actual by decile)

□ Feature importance stability
  - Check: Are the same features driving predictions?
  - Alert if: Top-5 features changed dramatically

---

RETRAIN TRIGGERS

□ Scheduled: Every [X] months
  YOUR SCHEDULE: _____________

□ Triggered: When any of these occur:
  - Precision@500 drops >___% from baseline
  - Feature drift detected on >___ features
  - Major product/business change
  
  YOUR THRESHOLDS: _____________

---

ESCALATION

Alert channel: _____________
Primary owner: _____________
Backup owner: _____________

"""
print(monitoring_plan)

## 3.2 Feedback Loop Awareness

Your model changes behavior → behavior changes data → data changes the next model.

In [ ]:
# TODO: Identify feedback loop risks

feedback_analysis = """
=== FEEDBACK LOOP ANALYSIS ===

The churn model targets high-risk customers for retention calls.
Some of them stay because of the call.

QUESTION 1: How does this affect your next training dataset?
(Think: what label do "saved" customers get?)

YOUR ANSWER: _____________________________________________________________
_____________________________________________________________

QUESTION 2: Could this make your next model worse? How?

YOUR ANSWER: _____________________________________________________________
_____________________________________________________________

QUESTION 3: What's one way to mitigate this risk?
(Hint: think about your experiment design from Part 1)

YOUR ANSWER: _____________________________________________________________
_____________________________________________________________

"""
print(feedback_analysis)

---

# Part 4: Final Deliverable - PM Update

Write a 150-250 word update for leadership explaining:
1. How you'll prove the model works (experiment plan)
2. A key risk with pure risk-based targeting
3. How you'll keep the model healthy (monitoring)

In [ ]:
# TODO: Write your PM update

pm_update = """
=== PM UPDATE: MODEL VALIDATION & MONITORING PLAN ===

Hi [Leadership Team],

[PROVING IMPACT]
TODO: Explain how you'll test whether the model actually reduces churn.
Include: experiment design, key metric, timeline.



[KEY RISK]
TODO: Explain one risk with targeting based purely on churn risk.
What could go wrong? How will you mitigate it?



[KEEPING IT HEALTHY]
TODO: Summarize your monitoring plan.
What will you check? When will you retrain?



[NEXT STEPS]
TODO: List 2-3 concrete actions you'll take.



Let me know if you have questions.

[Your Name]
"""

print(pm_update)
print(f"\nWord count: {len(pm_update.split())} words (target: 150-250)")

---

# Self-Assessment Checklist

In [ ]:
checklist = """
=== ADD-ON CHECKLIST ===

Part 1: Experiment Design
[ ] Completed experiment plan template
[ ] Chose appropriate Treatment/Control split
[ ] Estimated required duration
[ ] Identified guardrails

Part 2: Risk vs Uplift
[ ] Explained the Four Quadrants
[ ] Calculated break-even probability
[ ] Understood why risk ≠ value
[ ] Considered hybrid scoring approach

Part 3: Monitoring Plan
[ ] Defined daily/weekly/monthly checks
[ ] Set specific thresholds
[ ] Specified retrain triggers
[ ] Identified feedback loop risks

Part 4: PM Update
[ ] Explained experiment plan clearly
[ ] Communicated key risk
[ ] Summarized monitoring approach
[ ] Listed concrete next steps
[ ] 150-250 words, no jargon
"""
print(checklist)

---

## Congratulations!

You've completed the impact validation add-on. You now understand:

1. **How to prove business impact** with experiments, not just offline metrics
2. **Why risk models can waste money** and when to consider uplift signals
3. **How to keep a model healthy** with monitoring and retraining

These skills separate "ML project" from "ML in production." Well done!